In [45]:
 !pip install tweepy 

In [46]:
!pip install openpyxl 

In [47]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler

In [48]:
consumer_key="eTAbvNAoTvq2oWYF4pJEdX5gQ"

consumer_secret="batX46LNjH0iAcXGfaPQx3lDCAWhjq1GanICfmPPtuduUrKLWM"

access_token="1254039319930646532-PTszt9bW93VzEEr1yISzpNA8Jqqopo"

access_token_secret="wPSimV12XR5xP3MM8xvaonl6lVefgHblqMyZvH8vpwVMr"

In [49]:
auth=tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api=tweepy.API(auth)

In [50]:
query='Covid cases in India'

In [51]:
Tweets=api.search(query, count=10, language='en', exclude="retweets", tweet_mode='extended')

for tweet in Tweets:
    print(tweet,"\n")
    print(".......................................\n")




Status(_api=<tweepy.api.API object at 0x000001FB13C61100>, _json={'created_at': 'Thu Nov 19 01:01:28 +0000 2020', 'id': 1329228264863277056, 'id_str': '1329228264863277056', 'full_text': '#COVID__19\nTotal corona cases 19.11.20\nin India and in last 24 Hours\nConfirmed. 89,58,143. ( 45,369 )\nActive.          4,42,739\nRecovered. 83,81,770. ( 48,675 )\nDeceased.     1,31,618. ( 586)\nRecovery  rate  93.5 %\n#coronakoharanahai', 'truncated': False, 'display_text_range': [0, 233], 'entities': {'hashtags': [{'text': 'COVID__19', 'indices': [0, 10]}, {'text': 'coronakoharanahai', 'indices': [215, 233]}], 'symbols': [], 'user_mentions': [], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 71632

In [52]:
##Get the tweets and some attributes

In [53]:
df=pd.DataFrame(columns=['Tweets', 'User', 'User_statuses_count','User_followers', 'User_location', 'User_verified',
                        'fav_count', 'rt_count','tweet_date'])

In [54]:
df

,Tweets,User,User_statuses_count,User_followers,User_location,User_verified,fav_count,rt_count,tweet_date


In [55]:
def stream(data,file_name):
    i=0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i,'Tweets']=tweet.text
        df.loc[i, 'User']=tweet.user.name
        df.loc[i, 'User_statuses_count']=tweet.user.statuses_count
        df.loc[i, 'User_followers']=tweet.user.followers_count
        df.loc[i, 'User_location']=tweet.user.location
        df.loc[i, 'User_verified']=tweet.user.verified
        df.loc[i, 'fav_count']=tweet.favorite_count
        df.loc[i, 'rt_count']=tweet.retweet_count
        df.loc[i, 'tweet_date']=tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i=i+1
        if i==1000:
            break
        else:
            pass
        

In [56]:
stream(data=['Covid cases in India'], file_name='my_tweets')

In [57]:
df.head()

,Tweets,User,User_statuses_count,User_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,"Coronavirus India LIVE Updates: India Sees 38,...",Pehal News,84219,443,,False,0,0,2020-11-19 01:11:03
1,RT @timesofindia: India shows jump of 30% in n...,Mohammed Hussain,1853,156,Bangalore,False,0,38,2020-11-19 01:04:37
2,#COVID__19\nTotal corona cases 19.11.20\nin In...,M R Laroia,4150,56,"Gurgaon, India",False,0,0,2020-11-19 01:01:28
3,Random COVID-19 tests of 165 persons on Noida-...,India News Live,22976,794,India,False,0,0,2020-11-19 01:01:12
4,"India Sees 38,617 New Covid Infections; Delhi ...",Today News Headlines,10875,208,"New Delhi, India",False,0,0,2020-11-19 00:49:48


In [58]:
!pip install textblob


In [59]:
#Sentiment(polarity, subjectivily)

# Polarity score: [+1, -1]

# Subjectively : [0.0, 1.0]

#where 0 is very objective and 1 is very subjective

# Cleaning data 

In [60]:
from textblob import TextBlob

In [61]:
import re

def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)'," ", tweet).split())

In [62]:
def analyze_sentiment(tweet):
    analysis= TextBlob(tweet)
    if analysis.sentiment.polarity>0:
        return 'Positive'
    elif analysis.sentiment.polarity==0:
        return 'Negative'

In [63]:
df['clean_tweet']=df['Tweets'].apply(lambda x: clean_tweet(x))

In [64]:
df.head(2)

,Tweets,User,User_statuses_count,User_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet
0,"Coronavirus India LIVE Updates: India Sees 38,...",Pehal News,84219,443,,False,0,0,2020-11-19 01:11:03,Coronavirus India LIVE Updates India Sees 38 6...
1,RT @timesofindia: India shows jump of 30% in n...,Mohammed Hussain,1853,156,Bangalore,False,0,38,2020-11-19 01:04:37,RT India shows jump of 30 in new Covid cases s...


In [65]:
df['Sentiment']=df['clean_tweet'].apply(lambda x: analyze_sentiment(x))

df.head()

,Tweets,User,User_statuses_count,User_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,"Coronavirus India LIVE Updates: India Sees 38,...",Pehal News,84219,443,,False,0,0,2020-11-19 01:11:03,Coronavirus India LIVE Updates India Sees 38 6...,Positive
1,RT @timesofindia: India shows jump of 30% in n...,Mohammed Hussain,1853,156,Bangalore,False,0,38,2020-11-19 01:04:37,RT India shows jump of 30 in new Covid cases s...,Positive
2,#COVID__19\nTotal corona cases 19.11.20\nin In...,M R Laroia,4150,56,"Gurgaon, India",False,0,0,2020-11-19 01:01:28,COVID 19 Total corona cases 19 11 20 in India ...,Positive
3,Random COVID-19 tests of 165 persons on Noida-...,India News Live,22976,794,India,False,0,0,2020-11-19 01:01:12,Random COVID 19 tests of 165 persons on Noida ...,None
4,"India Sees 38,617 New Covid Infections; Delhi ...",Today News Headlines,10875,208,"New Delhi, India",False,0,0,2020-11-19 00:49:48,India Sees 38 617 New Covid Infections Delhi C...,Positive


In [67]:
n=204

print('Original_tweet: \n',df['Tweets'][n])
print()
print('sentiment of the tweet: \n', df['Sentiment'][n])
print()
print('Clean tweet: \n',df['clean_tweet'][n])

Original_tweet: 
 RT @brarharry685: We know that there are a lot of cases in India but we are safe right now. International students can do covid- 19 test be…

sentiment of the tweet: 
 Positive

Clean tweet: 
 RT We know that there are a lot of cases in India but we are safe right now International students can do covid 19 test be
